In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESF;
Init();


load the shock capturing bowShock


In [5]:
var database = OpenOrCreateDatabase("bosss_db_levelSets");

In [6]:
var sessions = database.Sessions;
sessions

#0: IBMBowShock_P2	IBMBowShock_p2_xCells40_yCells160_CFLFrac0.1_RK1_s0=1.0E-03_lambdaMax15	5/7/2020 8:33:16 AM	9c45ebf9...


In [42]:
var flds =sessions.Pick(0).Timesteps.Last().Fields.ToList();
flds

index,value
0,levelSet_recon
1,levelSet_recon_prc
2,levelSet_recon_prc_cont
3,rho
4,m0
5,m1
6,rhoE


In [43]:
var consVars= flds.GetRange(3,4).ToArray();
consVars

index,value
0,rho
1,m0
2,m1
3,rhoE


load the XDG BowShock to obtain the boundary level set

In [44]:
MatrixAssembler aaa = new MatrixAssembler(12);
BoSSSshell.WorkflowMgm.Init("XESF_BowShock_ConvStudy");
var dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path;
var db= OpenOrCreateDatabase(dbPath);
var levelSet = (SinglePhaseField) db.Sessions.Pick(0).Timesteps.Last().GetField("levelSet");

Project name is set to 'XESF_BowShock_ConvStudy'.
Default Execution queue is chosen for the database.


get material parameters

In [45]:
var control = XESF.XESFHardCodedControl.XDGBS_Local();
var material = control.GetMaterial();

transform to grid of shock capturing simulation

In [46]:
var lS_onSCGrid = (SinglePhaseField) flds.Pick(0).CloneAs();
lS_onSCGrid.Clear();
lS_onSCGrid.Identification="levelSet";
lS_onSCGrid.ProjectFromForeignGrid(1.0,levelSet);
flds.Add(lS_onSCGrid)

compute entropy and enthalpy field for the shock capturing simulation

In [47]:
using BoSSS.Solution.CompressibleFlowCommon;
var enthalpy = (SinglePhaseField) flds.Pick(3).CloneAs();
enthalpy.Clear();
enthalpy.Identification="enthalpy";
var entropy = (SinglePhaseField) flds.Pick(3).CloneAs();
entropy.Clear();
entropy.Identification="entropy";
enthalpy.ProjectFunction(1.0,
                        (X, U, j) => new StateVector(U, material).Enthalpy,new CellQuadratureScheme(true,CellMask.GetFullMask(enthalpy.GridDat)),
                        consVars);
entropy.ProjectFunction(1.0,
    (X, U, j) => new StateVector(U, material).Entropy,new CellQuadratureScheme(true,null),
    consVars);
enthalpy.GetExtremanlValues(....)
enthalpy.Coordinates.CheckForNanAndInf(...)
flds.Add(entropy);
flds.Add(enthalpy);
flds


index,value
0,levelSet_recon
1,levelSet_recon_prc
2,levelSet_recon_prc_cont
3,rho
4,m0
5,m1
6,rhoE
7,levelSet
8,entropy
9,enthalpy


plot everything

In [48]:
var tp = new Tecplot(enthalpy.GridDat,2);
tp.PlotFields("bowShockSC",0.0,flds);

## Enthalpy error
enthalpy is computed as
$$h=\frac{(\rho E + p)}{\rho},~ p=(c_P -1)(\rho E - k),~k=c_P \text{Ma}^2\frac{\rho }{2}(\vert u\vert^2 + \vert v\vert^2)$$


For an uniform inviscid flow the inflow enthalpy should remain constant throughout the flowfield 

In the case of the BowShock presented, the v-velocity is zero and we have
$$ h_{in} = \frac{\gamma}{\gamma-1} + \frac{1}{2}u_{in}^2 $$
We prescribe the inflow conditions in terms of pressure $p_{in}=1$, density $\rho_{in}=1$ and the Mach number $M_{in}=2$. From those the velocity is computed as 
$$ u_{in} = M_{in} \sqrt{\gamma \frac{p_{in}}{\rho_{in}}}= 2\sqrt{\gamma}$$

In [24]:
using BoSSS.Solution.CompressibleFlowCommon.MaterialProperty;
var enthalpy_end = si.Timesteps.Last().GetField("h");
var enthalpy_exact = (DGField) enthalpy_end.Clone();
double gamma = IdealGas.Air.HeatCapacityRatio;
double Ms = 2.0;
double densityLeft = 1;
double pressureLeft = 1;
double velocityXLeft = Ms * Math.Sqrt(gamma * pressureLeft / densityLeft);

(gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft

6.300000000000001

Compute the enthalpy error

In [25]:
var enthalpy_inflow = (gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft;
enthalpy_exact.Clear();
enthalpy_exact.ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.Identification= "h_exact";
var EEN = enthalpy_exact.L2Norm();
var EE = enthalpy_end.L2Error(enthalpy_exact)/EEN;
EE

0.22081460140149398